In [9]:
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from dash import dash_table 
import dash
from dash import dcc, html
from dash_table import DataTable
import plotly.express as px
from dash.dash_table.Format import Group
from IPython.display import display
from dash import dcc, html
import warnings
from dash_table import DataTable
import seaborn as sns
import numpy as np


In [2]:
data = pd.read_csv(r"C:\Users\harip\OneDrive\Documents\plum\DA Assignment - Sheet1.csv")

In [4]:
data['First_reply_time_in_minutes_within_business_days'] = data['First reply time in minutes within business hours']/(24*60)
data['First_resolution_in_days'] = data['First resolution time in minutes']/(24*60)
data['First_resolution_time_in_minutes_within_business_days'] = data['First resolution time in minutes within business hours']/(24*60)
data['Full_resolution_time_in_days'] = data['Full resolution time in minutes']/(24*60)
data['Full_resolution_time_in_minutes_within_business_days'] = data['Full resolution time in minutes within business hours']/(24*60)
data['Requester_wait_time_in_days'] = data['Requester wait time in minutes']/(24*60)
data['Requester_wait_time_in_minutes_within_business_days'] = data['Requester wait time in minutes within business hours']/(24*60)

In [5]:
data.drop(['First reply time in minutes within business hours','First resolution time in minutes','First resolution time in minutes within business hours','Full resolution time in minutes','Full resolution time in minutes within business hours',
       'Requester wait time in minutes',
       'Requester wait time in minutes within business hours'], axis = 1, inplace= True)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16476 entries, 0 to 16475
Data columns (total 23 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Id                                                     16476 non-null  int64  
 1   Requester id                                           16476 non-null  int64  
 2   Group                                                  16476 non-null  object 
 3   Status                                                 16476 non-null  object 
 4   Priority                                               16476 non-null  object 
 5   Via                                                    16476 non-null  object 
 6   Created at                                             16476 non-null  object 
 7   Updated at                                             16476 non-null  object 
 8   Assigned at                                   

In [7]:
data.isnull().sum()

Id                                                           0
Requester id                                                 0
Group                                                        0
Status                                                       0
Priority                                                     0
Via                                                          0
Created at                                                   0
Updated at                                                   0
Assigned at                                                129
Initially assigned at                                      129
Solved at                                                 2067
Resolution time                                           2067
Satisfaction Score                                       11023
Reopens                                                      0
Replies                                                      0
Manual Tagging of Categories [list]                    

In [17]:
data = data.replace('-', np.nan)

In [13]:
data['Manual Tagging of Categories [list]'].unique()

array(['Is my treatment covered (IMTC)', nan, 'Claims', 'Health ID',
       'Others', 'Not Applicable', 'Network or Blacklisted Hospitals',
       'Manage dependents', 'Telehealth', 'HR Queries', 'Health Benefits',
       'Super Top-up (STU)'], dtype=object)

In [15]:
data['Manual Tagging of Categories [list]'].fillna('Others')

0        Is my treatment covered (IMTC)
1                                Others
2                                Claims
3                                Claims
4                                Claims
                      ...              
16471                            Others
16472    Is my treatment covered (IMTC)
16473                            Others
16474                            Others
16475                            Others
Name: Manual Tagging of Categories [list], Length: 16476, dtype: object

In [16]:
# Convert time columns to datetime
time_columns = ['Created at', 'Updated at', 'Assigned at', 'Initially assigned at', 'Solved at']
for column in time_columns:
    data[column] = pd.to_datetime(data[column], errors='coerce')

# Calculate time to resolve
data['Time_to_Resolve'] = (data['Solved at'] - data['Created at']).dt.total_seconds() / (60*60*24)  # Convert to minutes

# Group by ticket type
grouped_by_type = data.groupby('Manual Tagging of Categories [list]')

# Calculate average time to resolve for each ticket type
average_time_by_type = grouped_by_type['Time_to_Resolve'].mean().reset_index()

# Dash app
app = dash.Dash(__name__)

# Layout of the Dash app
app.layout = html.Div(children=[
    # Bar chart for ticket types
    dcc.Graph(
        id='bar-chart-type',
        figure={
            'data': [
                go.Bar(
                    x=average_time_by_type['Manual Tagging of Categories [list]'],
                    y=average_time_by_type['Time_to_Resolve'],
                    text=average_time_by_type['Time_to_Resolve'].round(2),
                    textposition='outside',
                    marker=dict(color='skyblue'),
                )
            ],
            'layout': {
                'title': 'Average Time to Resolve by Ticket Type',
                'xaxis': {'title': 'Ticket Type'},
                'yaxis': {'title': 'Average Time to Resolve (minutes)'},
            }
        }
    ),
    # DataTable for ticket types
    DataTable(
        id='table-type',
        columns=[
            {"name": col, "id": col} for col in data.groupby('Manual Tagging of Categories [list]')['Time_to_Resolve'].describe().reset_index().columns
        ],
        data=data.groupby('Manual Tagging of Categories [list]')['Time_to_Resolve'].describe().reset_index().to_dict('records'),
    ),
])

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)


OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.